In [23]:
!pip install torch torchvision
!pip install -U git+https://github.com/luca-medeiros/lang-segment-anything.git

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/luca-medeiros/lang-segment-anything.git to /tmp/pip-req-build-f4fh3yra
  Running command git clone --filter=blob:none --quiet https://github.com/luca-medeiros/lang-segment-anything.git /tmp/pip-req-build-f4fh3yra
  Resolved https://github.com/luca-medeiros/lang-segment-anything.git to commit 134a48a81ebd691686187ecce2d016fb327a8852
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/IDEA-Research/GroundingDINO.git to /tmp/pip-install-lzhjdvb3/groundingdino_de748bcf1e1945018784618d4df2f8ad
  Running command git clone --filter=blob:none --quiet https://github.com/IDEA-Research/GroundingDINO.git /tmp/pip-install-lzhjdvb3/groundingdino_de748bcf1e1945018784618d4df2f8ad
  Resolved https://github.com/IDEA-Research/GroundingDINO.git to commit 2b62f419c292ca9c518daae55512fabc3fead4a4
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/fac

In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [25]:
import warnings
warnings.filterwarnings("ignore")

import os
import json
import uuid
import random
import glob

import numpy as np
import cv2
from PIL import Image
from matplotlib import pyplot as plt

from lang_sam import LangSAM

In [26]:
def draw_bounding_boxes(image_path, label_path):
    
    """
        draw bounding boxes by passing imagepath and label file path,
        expects yolo type bbox annotations in the labelfile.
    """
    
    def clamp(val, min_val, max_val):
        return max(min_val, min(max_val, val))

    # Load image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image: {image_path}")
        return
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    fig, ax = plt.subplots()
    ax.imshow(image)
    ax.set_title("Image with Bounding Boxes")
    ax.axis('off')

    label_file=label_path

    

    with open(label_file, 'r') as f:
        for line in f:
            class_id, x_center, y_center, width, height = map(float, line.split())
            
            # Clamp values to be within the image
            x_center, y_center, width, height = map(lambda x: clamp(x, 0, 1), [x_center, y_center, width, height])

            # Convert normalized coordinates to absolute coordinates
            abs_x_center = int(x_center * image.shape[1])
            abs_y_center = int(y_center * image.shape[0])
            abs_width = int(width * image.shape[1])
            abs_height = int(height * image.shape[0])

            # Calculate top left corner coordinates
            x1 = int(abs_x_center - abs_width / 2)
            y1 = int(abs_y_center - abs_height / 2)


            # Clamp top left corner coordinates to be within the image
            x1 = clamp(x1, 0, image.shape[1] - 1)
            y1 = clamp(y1, 0, image.shape[0] - 1)

            # Adjust width and height if bounding box exceeds image boundaries
            abs_width = min(abs_width, image.shape[1] - x1)
            abs_height = min(abs_height, image.shape[0] - y1)

    

            # Draw bounding box
            rect = plt.Rectangle((x1, y1), abs_width, abs_height, fill=False, edgecolor='red', linewidth=2)
            ax.add_patch(rect)

            # Print bounding box coordinates on the image
            ax.text(x1, y1, f"{int(class_id)}, ({x1}, {y1})", fontsize=8, verticalalignment='top')

In [27]:
def display_image_with_masks(image, masks):
    num_masks = len(masks)

    fig, axes = plt.subplots(1, num_masks + 1, figsize=(15, 5))
    axes[0].imshow(image)
    axes[0].set_title("Original Image")
    axes[0].axis('off')

    for i, mask_np in enumerate(masks):
        axes[i+1].imshow(mask_np, cmap='gray')
        axes[i+1].set_title(f"Mask {i+1}")
        axes[i+1].axis('off')

    plt.tight_layout()
    plt.show()

def display_image_with_boxes(image, boxes, logits):
    fig, ax = plt.subplots()
    ax.imshow(image)
    ax.set_title("Image with Bounding Boxes")
    ax.axis('off')

    for box, logit in zip(boxes, logits):
        x_min, y_min, x_max, y_max = box
        confidence_score = round(logit.item(), 2)  # Convert logit to a scalar before rounding
        box_width = x_max - x_min
        box_height = y_max - y_min

        # Draw bounding box
        rect = plt.Rectangle((x_min, y_min), box_width, box_height, fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)

        # Add confidence score as text
        ax.text(x_min, y_min, f"Confidence: {confidence_score}", fontsize=8, color='red', verticalalignment='top')

    plt.show()

    
def print_bounding_boxes(boxes):
    print("Bounding Boxes:")
    for i, box in enumerate(boxes):
        print(f"Box {i+1}: {box}")

def print_detected_phrases(phrases):
    print("\nDetected Phrases:")
    for i, phrase in enumerate(phrases):
        print(f"Phrase {i+1}: {phrase}")

def print_logits(logits):
    print("\nConfidence:")
    for i, logit in enumerate(logits):
        print(f"Logit {i+1}: {logit}")
        
def show_annotations(image_path, prompt):
    image_pil = Image.open(image_path).convert("RGB")
    masks, boxes, phrases, logits = model.predict(image_pil, prompt)
    if len(masks) == 0:
        print(f"no masks found for {prompt}!")
        return
    masks_np = [mask.squeeze().cpu().numpy() for mask in masks]
    display_image_with_boxes(image_pil, boxes, logits)
    
model = LangSAM()

final text_encoder_type: bert-base-uncased
Model loaded from /root/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])


In [7]:
os.getcwd()

'/kaggle/working'

In [28]:
# if os.path.exists('./boattypesdata'):
#     os.system('rm -rf ./boattypesdata/')

if os.path.exists('./shipdataset2'):
    os.system('rm -rf ./shipdataset2/')
    
# os.mkdir('./boattypesdata')
os.mkdir('./shipdataset2')

# data_sam/boattypes/images/
# data_sam/boattypes/labels/

# image_copy_location = os.path.join(os.getcwd(), 'boattypesdata', 'images/')
# label_dir_location = os.path.join(os.getcwd(), 'boattypesdata', 'labels/')

# if os.path.exists(image_copy_location):
#     os.system(f'rm -rf {image_copy_location}')

# if os.path.exists(label_dir_location):
#     os.system(f'rm -rf {label_dir_location}')


# os.mkdir(os.path.join(os.getcwd(), 'boattypesdata', 'images'))
# os.mkdir(os.path.join(os.getcwd(), 'boattypesdata', 'labels'))

# # data_sam/shipdataset2/images/
# # data_sam/shipdataset2/labels/
os.mkdir(os.path.join(os.getcwd(), 'shipdataset2', 'images'))
os.mkdir(os.path.join(os.getcwd(), 'shipdataset2', 'labels'))

In [50]:
# path to actual data
boatdatapath = '/kaggle/input/boattypes/Boattypes/'

# prepare prompts
allclasses = glob.glob(os.path.join(boatdatapath, '*'))
class2prompt = {}
num_images = {}
for class1 in allclasses:
    classname = class1.split('/')[-1]
    
    if classname=='boats':
        continue
    
    if 'buoy' in classname:
        class2prompt[classname] = 'buoy'
    else:
        class2prompt[classname] = 'boat'
    num_images[classname] = len(glob.glob(os.path.join(boatdatapath, classname, '*')))
class2prompt

{'inflatable boat': 'boat',
 'gondola': 'boat',
 'freight boat': 'boat',
 'cruise ship': 'boat',
 'ferry boat': 'boat',
 'kayak': 'boat',
 'buoy': 'buoy',
 'sailboat': 'boat'}

In [51]:
sum([num_images[k] for k in num_images]), allclasses

(1420,
 ['/kaggle/input/boattypes/Boattypes/inflatable boat',
  '/kaggle/input/boattypes/Boattypes/gondola',
  '/kaggle/input/boattypes/Boattypes/freight boat',
  '/kaggle/input/boattypes/Boattypes/cruise ship',
  '/kaggle/input/boattypes/Boattypes/ferry boat',
  '/kaggle/input/boattypes/Boattypes/kayak',
  '/kaggle/input/boattypes/Boattypes/buoy',
  '/kaggle/input/boattypes/Boattypes/boats',
  '/kaggle/input/boattypes/Boattypes/sailboat'])

In [52]:
image_copy_location = os.path.join(os.getcwd(), 'boattypesdata', 'images/')
label_dir_location = os.path.join(os.getcwd(), 'boattypesdata', 'labels/')

image_copy_location, label_dir_location

('/kaggle/working/boattypesdata/images/',
 '/kaggle/working/boattypesdata/labels/')

In [55]:

for class1 in ['sailboat']:
    classname = class1.split('/')[-1]
    textprompt = class2prompt[classname]
    
    print(f'labelling for class: "{classname}" with prompt: "{textprompt}"')
    label = 0 if textprompt=='boat' else 1
    
    image_paths = glob.glob(os.path.join(boatdatapath, classname, '*'))
    print(f'total images={len(image_paths)}')
    
    for _, imagepath in enumerate(image_paths):
        labelfile = imagepath.split('/')[-1].split('.')[0] + '.txt'
        labelfilepath = os.path.join(label_dir_location, labelfile)
        #print(imagepath)
        #print(labelfilepath)
        
        # langsam annotation code starts
        image_pil = Image.open(imagepath).convert("RGB")
        masks, boxes, phrases, logits = model.predict(image_pil, textprompt)
        if len(masks) == 0:
            print(f"no masks found for {textprompt} for {imagepath}!")
            continue
        #masks_np = [mask.squeeze().cpu().numpy() for mask in masks]
        #display_image_with_boxes(image_pil, boxes, logits)
        #print_logits(logits)
        # langsam annotation code ends
        
        
        # create label files from annotations
        H, W = image_pil.size[1], image_pil.size[0]
        
        to_write = []
        for box in boxes:
            x_min, y_min, x_max, y_max = box.numpy()
            #print(x_min, y_min)
            #print(x_max, y_max)

            x_c, y_c = (x_min + x_max) / 2, (y_min + y_max) / 2
            box_width = x_max - x_min
            box_height = y_max - y_min

            x_c, y_c = x_c/W, y_c/H
            box_width = box_width/W
            box_height = box_height/H

            s = f"{label} {x_c:.6} {y_c:.6} {box_width:.6} {box_height:.6}\n"
            # print(s)
            to_write.append(s)
            
        with open(labelfilepath, "w") as f:
            for s in to_write:
                f.write(s)
                
#         if _ >= 10:
#             break


#     break

labelling for class: "sailboat" with prompt: "boat"
total images=487


In [56]:
classname = list(class2prompt.keys())
print(classname)

['inflatable boat', 'gondola', 'freight boat', 'cruise ship', 'ferry boat', 'kayak', 'buoy', 'sailboat']


In [ ]:
image_paths = glob.glob(os.path.join(boatdatapath, 'gondola', '*'))
label_paths = glob.glob(os.path.join(label_dir_location, '*'))

## download to local

In [59]:
os.getcwd()

'/kaggle/working'

In [61]:
from IPython.display import FileLink

In [64]:
# !zip -r boattypesdata.zip /kaggle/working/boattypesdata/
FileLink(r'./boattypesdata.zip')

In [65]:
FileLink(r'./boattypesdata.zip')

/kaggle/working/boattypesdata.zip

In [1]:
num_images

NameError: name 'num_images' is not defined

In [10]:
!ls /kaggle/input/boattypes/Boattypes/

 boats	'cruise ship'  'freight boat'  'inflatable boat'   sailboat
 buoy	'ferry boat'    gondola         kayak


In [ ]:
! cp /kaggle/input/yolo-code/YOLOv7-DL23 ./ -r

In [35]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Tue Jan  2 11:53:15 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    36W / 250W |   8695MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [29]:
shipdatapath = '/kaggle/input/shipdataset2/'
image_paths = glob.glob(os.path.join(shipdatapath, 'images', '*'))

In [30]:
label_dir_location = label_dir_location = os.path.join(os.getcwd(), 'shipdataset2', 'labels/')

In [36]:
for _, imagepath in enumerate(image_paths):
    labelfile = imagepath.split('/')[-1].split('.')[0] + '.txt'
    labelfilepath = os.path.join(label_dir_location, labelfile)
    if labelfilepath in label_paths:
        continue
#     print(imagepath)
#     print(labelfilepath)

    # langsam annotation code starts
    image_pil = Image.open(imagepath).convert("RGB")
    masks, boxes, phrases, logits = model.predict(image_pil, 'boat')
    if len(masks) == 0:
        print(f"no masks found for boat for {imagepath}!")
        continue
#     masks_np = [mask.squeeze().cpu().numpy() for mask in masks]
#     display_image_with_boxes(image_pil, boxes, logits)
    #print_logits(logits)
    # langsam annotation code ends


    # create label files from annotations
    H, W = image_pil.size[1], image_pil.size[0]

    to_write = []
    for box in boxes:
        x_min, y_min, x_max, y_max = box.numpy()
        #print(x_min, y_min)
        #print(x_max, y_max)
        x_c, y_c = (x_min + x_max) / 2, (y_min + y_max) / 2
        box_width = x_max - x_min
        box_height = y_max - y_min
        x_c, y_c = x_c/W, y_c/H
        box_width = box_width/W
        box_height = box_height/H
        s = f"{0} {x_c:.6} {y_c:.6} {box_width:.6} {box_height:.6}\n"
        # print(s)
        to_write.append(s)

    with open(labelfilepath, "w") as f:
        for s in to_write:
            f.write(s)

no masks found for boat for /kaggle/input/shipdataset2/images/1124945.jpg!
no masks found for boat for /kaggle/input/shipdataset2/images/2874063.jpg!
no masks found for boat for /kaggle/input/shipdataset2/images/2821097.jpg!
no masks found for boat for /kaggle/input/shipdataset2/images/2827132.jpg!
no masks found for boat for /kaggle/input/shipdataset2/images/2814960.jpg!
no masks found for boat for /kaggle/input/shipdataset2/images/2800540.jpg!
no masks found for boat for /kaggle/input/shipdataset2/images/2837457.jpg!
no masks found for boat for /kaggle/input/shipdataset2/images/2784215.jpg!
no masks found for boat for /kaggle/input/shipdataset2/images/2800437.jpg!
no masks found for boat for /kaggle/input/shipdataset2/images/2794063.jpg!
no masks found for boat for /kaggle/input/shipdataset2/images/924996.jpg!
no masks found for boat for /kaggle/input/shipdataset2/images/2640669.jpg!
no masks found for boat for /kaggle/input/shipdataset2/images/2800489.jpg!
no masks found for boat fo

In [37]:
label_dir_location

'/kaggle/working/shipdataset2/labels/'

In [38]:
label_paths = glob.glob(os.path.join(label_dir_location, '*'))

In [39]:
len(label_paths)

8911

In [40]:
shipdatapath

'/kaggle/input/shipdataset2/'

In [43]:
from IPython.display import FileLink

In [44]:
# !zip -r shipdataset2.zip /kaggle/working/shipdataset2/
FileLink(r'./shipdataset2.zip')

/kaggle/working/shipdataset2.zip